In [ ]:
import cv2
import numpy as np
import face_recognition
from PIL import Image as PIL_Image
from pynq import Overlay
from pynq import GPIO
from IPython.display import clear_output as clear
import matplotlib.pyplot as plt
import time
from pynq.lib.video import *

In [ ]:
displayport = DisplayPort()  # 设置视频输出端口与分辨率
displayport.configure(VideoMode(1280, 720, 24), PIXEL_RGB)

ol = Overlay('/home/xilinx/pynq/overlays/GPIO/design_1.bit')
LED0 = GPIO(GPIO.get_gpio_pin(0),'out')
LED1 = GPIO(GPIO.get_gpio_pin(1),'out')
LED2 = GPIO(GPIO.get_gpio_pin(2),'out')
LED3 = GPIO(GPIO.get_gpio_pin(3),'out')

In [ ]:
# GPIO输出，1输出3.3V，0输出GND。共阳极0亮1灭
def GPIO_Voltage(val):
    LED0.write(val)
    LED1.write(val)
    LED2.write(val)
    LED3.write(val)

# 摄像头捕获图片
def picture_capture():
    capture = cv2.VideoCapture(0)                                                       # 设置视频捕获端口与分辨率
    capture.set(3, 1280)
    capture.set(4, 720)
    frame = displayport.newframe()                                                      # 新建空白帧
    capture.read(frame)                                                                 # 向空白帧写入数据——未处理图像
    capture.release()                                                                   # 关闭捕获端口
    return frame
    
# 人脸识别的主体
def faceRecognition():
    frame = picture_capture()
    
    frameRGB = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2RGB)
    facesLocate = face_recognition.face_locations(frameRGB)                             # 对摄像头读取的检测人脸
    faceEncoded = face_recognition.face_encodings(frameRGB, facesLocate)                # 进行特征编码
    
    name = 'unknow'
    lab = 'unknow'
    color1 = (0, 255, 0)                                                                # 初始化面部捕捉框显示绿色
    
    for (top, right, bottom, left), face_encoding in zip(facesLocate, faceEncoded):    # 遍历检测的人脸和库中读取的图片进行对比，计算其相似度
        matchs = face_recognition.compare_faces(EncodeingList, face_encoding)           # 进行匹配
        distance = face_recognition.face_distance(EncodeingList, face_encoding)         # 计算相似度
        for index, item in enumerate(distance):
            Similarity = 1 - item
            if Similarity > 0.5:
                if matchs[index]:
                    lab = 'name:{}; Similarity:{}'.format(classNames[index], Similarity)  # 得到匹配到的图片名称与相似度值
                    break
        
        cv2.rectangle(img=frame, pt1=(left, top), pt2=(right, bottom), color=color1, thickness=3)       # 绘制画面部捕捉框
        cv2.putText(frame, lab, (left, top - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color1, 2)              # 在捕捉框上添加匹配到的图片信息
        # cv2.imwrite('{}/{}.png'.format(picture_path, name), frame)                                      # 保存图片
    displayport.writeframe(frame)                                                                   # 输出图像帧——处理过的
    frame = []
    
    print(lab)                                                                          # 输出识别信息
    if lab is 'unknow':
        GPIO_Voltage(1)
    else:
        GPIO_Voltage(0)

In [ ]:
if __name__ == '__main__':
    EncodeingList = np.load('database/EncodeingList.npz')       # 导入数据库_______数据库所有图像编码集合
    EncodeingList = EncodeingList['arr_0']
    classNames = np.load('database/classNames.npz')             # 导入数据库_______数据库所有图像的名称（分类）
    classNames = classNames['arr_0']
    while 1:
        faceRecognition()                                       # 人脸识别